# Forecast of Weather Temperatures

## Problem Statement

We would like to forecast the temperatures if we know the temperature of the previous days

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

In [5]:
df = pd.read_csv("GlobalTemperatures.csv")
df=df[["dt","LandAverageTemperature"]]
df.columns=["date","t0"]

In [6]:
df.head()

,date,t0
0,1750-01-01,3.034
1,1750-02-01,3.083
2,1750-03-01,5.626
3,1750-04-01,8.490
4,1750-05-01,11.573


In [9]:
df["mes"] = pd.to_datetime(df["date"],format="%Y-%m-%d").dt.month

In [10]:
df.head()

,date,t0,mes
0,1750-01-01,3.034,1
1,1750-02-01,3.083,2
2,1750-03-01,5.626,3
3,1750-04-01,8.490,4
4,1750-05-01,11.573,5


In [11]:
# lagged values
input_col = []
for i in range(1,10):
    input_col.append("t"+str(i))
    df["t"+str(i)]=df["t0"].shift(i)

In [14]:
df.head()

,date,t0,mes,t1,t2,t3,t4,t5,t6,t7,t8,t9
0,1750-01-01,3.034,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1750-02-01,3.083,2,3.034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1750-03-01,5.626,3,3.083,3.034,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1750-04-01,8.490,4,5.626,3.083,3.034,NaN,NaN,NaN,NaN,NaN,NaN
4,1750-05-01,11.573,5,8.490,5.626,3.083,3.034,NaN,NaN,NaN,NaN,NaN


In [15]:
df=df.dropna()

In [16]:
df.head()

,date,t0,mes,t1,t2,t3,t4,t5,t6,t7,t8,t9
9,1750-10-01,6.367,10,11.413,14.750,15.868,12.937,11.573,8.490,5.626,3.083,3.034
42,1753-07-01,15.092,7,14.585,12.186,8.272,5.424,0.715,2.039,5.086,7.335,7.839
43,1753-08-01,14.148,8,15.092,14.585,12.186,8.272,5.424,0.715,2.039,5.086,7.335
44,1753-09-01,13.322,9,14.148,15.092,14.585,12.186,8.272,5.424,0.715,2.039,5.086
45,1753-10-01,8.962,10,13.322,14.148,15.092,14.585,12.186,8.272,5.424,0.715,2.039


In [17]:
# Historico de entrenamiento
df=df[df["date"]>"1950"]

In [18]:
# Hold-out
df_train = df[df["date"]<="2012"]
df_test = df[df["date"]>"2012"]

In [19]:
df.tail()

,date,t0,mes,t1,t2,t3,t4,t5,t6,t7,t8,t9
3187,2015-08-01,14.755,8,15.051,14.505,12.312,9.313,6.740,4.664,3.881,4.850,6.713
3188,2015-09-01,12.999,9,14.755,15.051,14.505,12.312,9.313,6.740,4.664,3.881,4.850
3189,2015-10-01,10.801,10,12.999,14.755,15.051,14.505,12.312,9.313,6.740,4.664,3.881
3190,2015-11-01,7.433,11,10.801,12.999,14.755,15.051,14.505,12.312,9.313,6.740,4.664
3191,2015-12-01,5.518,12,7.433,10.801,12.999,14.755,15.051,14.505,12.312,9.313,6.740


In [20]:
input_col

['t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9']

In [22]:
X_train=df_train[input_col+["mes"]]
y_train=df_train["t0"]

In [23]:
X_test=df_test[input_col+["mes"]]
y_test=df_test["t0"]

In [24]:
from sklearn.tree import DecisionTreeRegressor

In [25]:
reg = DecisionTreeRegressor(max_depth=5)
reg.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [26]:
from sklearn.metrics import mean_squared_error

In [27]:
mean_squared_error(y_test,reg.predict(X_test))

0.27902913764040926

In [28]:
from sklearn.ensemble import RandomForestRegressor
reg=RandomForestRegressor(n_estimators=50,max_depth=5)
reg.fit(X_train,y_train)
mean_squared_error(y_test,reg.predict(X_test))

0.2205264792935382